In [1]:
import socket

NB='test'
DESCRIPTION='test notebook'
HOST = socket.gethostname()
HOST

'DESKTOP-7KHT10A'

In [2]:
import warnings
import os
from pathlib import Path

import mlflow
from dataclasses import dataclass
from dataclasses import asdict

import pandas as pd
#import lightgbm as lgb
#import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, average_precision_score, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

import pickle

#from my_utils import get_notebook_path

def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)
        
def unpickle(filename):
    with open(filename, mode='rb') as fo:
        p = pickle.load(fo)
    return p 

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)

In [3]:
ROOT_DIR = Path('../')
DATA_DIR = ROOT_DIR / Path('data')

In [4]:
class Config:
    N_LABEL = 24
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 0.15
    PATIENCE = 30
    EPOCH = 300
    BATCH_SIZE = 6
    SKIP_EVALUATE_NUM = 5

In [5]:
train_df = pd.read_csv(DATA_DIR / Path('house_price_train.csv'))
test_df = pd.read_csv(DATA_DIR / Path('house_price_test.csv'))

display(train_df.shape)
display(train_df.head(5))
display(test_df.shape)
display(test_df.head(5))

(1460, 81)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


(1459, 80)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,...,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## feature and target

In [6]:
target = 'SalePrice'
del_columns = ['SalePrice', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

features = list(set(train_df.columns) - set(del_columns))
features.sort()

to_pickle(ROOT_DIR / Path(f'output/{HOST}_{NB}_features.pkl'), features)

print(features)

['MasVnrArea', 'YearRemodAdd', 'TotalBsmtSF', 'PoolArea', 'FullBath', 'MiscVal', 'KitchenAbvGr', 'GarageCars', 'Id', '3SsnPorch', 'BsmtFullBath', 'GarageArea', 'WoodDeckSF', '1stFlrSF', 'TotRmsAbvGrd', 'LotFrontage', 'HalfBath', '2ndFlrSF', 'LotArea', 'MoSold', 'YearBuilt', 'BsmtFinSF2', 'YrSold', 'MSSubClass', 'GrLivArea', 'BsmtFinSF1', 'OverallQual', 'GarageYrBlt', 'OverallCond', 'ScreenPorch', 'Fireplaces', 'BedroomAbvGr', 'BsmtUnfSF', 'EnclosedPorch', 'OpenPorchSF', 'LowQualFinSF', 'BsmtHalfBath']


## train lgbm

### metrix

In [7]:
def evaluation(true, pred):
    return np.sqrt(mean_squared_error(true, pred))

### encording

In [8]:
# 数値でないカラムをLabelEncording
categoricals = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
features = list(set(features) | set(categoricals))
for c in categoricals:
    le = LabelEncoder()
    le.fit(train_df.append(test_df)[c].astype(str))
    train_df[c] = le.transform(train_df[c].astype(str))
    test_df[c] = le.transform(test_df[c].astype(str))

# Count encoding
count_enc = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
count_features = []
for c in count_enc:
    _count = train_df.append(test_df)[c].astype(str).value_counts().to_dict()
    train_df[f'{c}_count'] = train_df[c].astype(str).map(_count)
    test_df[f'{c}_count'] = test_df[c].astype(str).map(_count)
    count_features.append(f'{c}_count')
features = list(set(features) | set(count_features))

### train

In [9]:
params = {
    'learning_rate': 0.01, 
    'loss_function': 'RMSE',
    'verbose':100,
    'iterations':5000,
    'early_stopping_rounds':100
}

# TargetEncording Config
target_enc = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'] # 集計対象カラム
target_enc_key = ['SalePrice'] # 集計するキー

oof_preds = np.zeros(len(train_df))
y_pred = np.zeros(len(test_df))
models = []
cv_scores = {}
skf = StratifiedKFold(n_splits=Config.N_FOLD, random_state=Config.RANDOM_SATE, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(train_df[features], train_df[target])):

    print(f'====== fold {fold} ======')

    # TrainとTestに分割
    x_train, x_val = train_df.copy().iloc[train_index][features], train_df.copy().iloc[test_index][features]
    y_train, y_val =  train_df.iloc[train_index][target], train_df.iloc[test_index][target]

    test = test_df[features]

    # Target Encoding
    if len(target_enc) > 0:
        for t in target_enc_key:
            for c in target_enc:
                x_train[f'{c}_target_enc_by_{t}'] = train_df.iloc[train_index][c].map(train_df.iloc[test_index].groupby(c)[t].mean().to_dict())
                x_val[f'{c}_target_enc_by_{t}'] = train_df.iloc[test_index][c].map(train_df.groupby(c)[t].mean().to_dict())
                test[f'{c}_target_enc_by_{t}'] = test_df[c].map(train_df.groupby(c)[t].mean().to_dict())
            
    train_features = x_train.columns.to_list()
    
    # create Dataset
    train_pool = Pool(x_train, y_train, cat_features=categoricals)
    valid_pool = Pool(x_val, y_val, cat_features=categoricals)
    test_pool = Pool(test, cat_features=categoricals) 
    
    # train
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=valid_pool)
    models.append(model)

    fold_pred = model.predict(valid_pool)
    
    score = evaluation(y_val, fold_pred)
    cv_scores[f'cv{fold}'] = score

    oof_preds[test_index] = fold_pred

    y_pred += model.predict(test_pool) / Config.N_FOLD

    print(f'cv score is {score}')

oof_score = evaluation(train_df[target], oof_preds)
print(f'OOF score is {oof_score}')

====== fold 0 ======
0:	learn: 79724.6822225	test: 75628.6111168	best: 75628.6111168 (0)	total: 221ms	remaining: 18m 25s
100:	learn: 46437.5361731	test: 45385.4340549	best: 45385.4340549 (100)	total: 4.65s	remaining: 3m 45s
200:	learn: 31886.6326973	test: 36095.7986535	best: 36095.7986535 (200)	total: 9.37s	remaining: 3m 43s
300:	learn: 25255.7145218	test: 33766.7071862	best: 33761.2014578 (299)	total: 14.2s	remaining: 3m 42s
400:	learn: 21915.1750990	test: 33829.9846620	best: 33573.8610912 (329)	total: 18.7s	remaining: 3m 34s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 33573.86109
bestIteration = 329

Shrink model to first 330 iterations.
cv score is 33573.86109117422
====== fold 1 ======
0:	learn: 79976.0503695	test: 74505.7882854	best: 74505.7882854 (0)	total: 40ms	remaining: 3m 20s
100:	learn: 46828.2897892	test: 46910.5751188	best: 46910.5751188 (100)	total: 5.13s	remaining: 4m 8s
200:	learn: 32692.7954348	test: 36140.4968685	best: 36140.4968685 (200)	total: